<a href="https://colab.research.google.com/github/gabelev/synthetic_vocal_identification/blob/main/process_vocals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Authorize me
from google.colab import auth
auth.authenticate_user()

In [ ]:
# %%capture
# Install some dependencies if in colab
!%apt install ffmpeg
%pip install spleeter
%pip install tdqm
%pip install torchaudio
%pip install librosa
%pip install safetensors
%pip install pyannote.audio
%pip install pyannote.core[notebook]
%pip install tensorflow==2.9.1
%pip install pyannote.pipeline==1.5.2


In [ ]:
!gsutil -m cp -r gs://singer-id/ai_vocal_id_files .

Copying gs://singer-id/ai_vocal_id_files/-Vo2vDsFdfw.mp3...
Copying gs://singer-id/ai_vocal_id_files/0xoPoN-Etgg.mp3...
Copying gs://singer-id/ai_vocal_id_files/5BfB3YRzMwo.mp3...
Copying gs://singer-id/ai_vocal_id_files/5IYpleg9hJk.mp3...
Copying gs://singer-id/ai_vocal_id_files/7QdxDJxJuAc.mp3...
Copying gs://singer-id/ai_vocal_id_files/C0Mja5SdZEk.mp3...
Copying gs://singer-id/ai_vocal_id_files/GjsSzqAWFAU.mp3...
Copying gs://singer-id/ai_vocal_id_files/HGV-H-mgK60.mp3...
Copying gs://singer-id/ai_vocal_id_files/IQscAY8-9vg.mp3...
Copying gs://singer-id/ai_vocal_id_files/Lr8HJ3oW9-8.mp3...
Copying gs://singer-id/ai_vocal_id_files/PWQG3mwH8i8.mp3...
Copying gs://singer-id/ai_vocal_id_files/SDR9GuT_86w.mp3...
Copying gs://singer-id/ai_vocal_id_files/U-y6K8X3hYM.mp3...
Copying gs://singer-id/ai_vocal_id_files/XENVZRVzQNQ.mp3...
Copying gs://singer-id/ai_vocal_id_files/YNy8VqlPX6Q.mp3...
Copying gs://singer-id/ai_vocal_id_files/ZSQBxafzkX0.mp3...
Copying gs://singer-id/ai_vocal_id_files

In [ ]:
# !mkdir music/

In [ ]:
# !tar -xf /content/singer-id/artist20-mp3s-32k.tgz -C music/

In [ ]:
!mkdir ai_vocal_id_tensors/

In [ ]:
HF_TOKEN = "hf_???"

In [ ]:
import librosa
import os
import torch
import torchaudio

import numpy as np

from pyannote.audio import Inference
from pyannote.audio import Model
from spleeter.separator import Separator
from tqdm import tqdm


model = Model.from_pretrained(
    "pyannote/embedding",
    use_auth_token=HF_TOKEN,
)
embedding_model = Inference(model, window="whole")
separator = Separator('spleeter:2stems')
sample_rate = 16_000


def pre_process_new(
            input_file,
            separator,
            embedding_model,
            sample_rate=16_000,
            remove_silences=True,  # used for training prep
            normalize=False,  # Probably always good to do
            mono=True,
            embedding=True,
            verbose=False,
    ):
        # Load audio
        # raw_audio, sr = torchaudio.load(input_file, channels_first=False)
        raw_audio, sr = librosa.load(input_file, sr=sample_rate, mono=mono)
        if verbose:
            print(f"Loaded audio {input_file}, sample_rate: {sr}")

        # Tony's Bug -- let's handle mono files
        if raw_audio.shape[0] != 2:
            raw_audio = np.array([raw_audio,raw_audio])

        raw_audio = np.swapaxes(raw_audio, 0, 1)

        # removed vocals
        prediction = separator.separate(raw_audio)
        vox = prediction["vocals"]
        #audio = vox

        audio = np.swapaxes(vox, 0, 1)
        # remove silences
        if remove_silences:
            no_silences = librosa.effects.trim(
                y=audio, frame_length=8000, top_db=40)
            audio = no_silences[0]

        if normalize:
            # normailze the volume
            audio = librosa.util.normalize(audio, axis=0)

        # convert to torch tensor
        audio = torch.from_numpy(audio)

        #make mono again?
        if mono:
            audio = torch.mean(audio, dim=0, keepdim=True)

        if embedding:
            embedding = embedding_model({"waveform": audio, "sample_rate": sample_rate})
            return embedding
        return audio

INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.2.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--embedding/snapshots/4db4899737a38b2d618bbd74350915aa10293cb2/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.2.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.2.1+cu121. Bad things might happen unless you revert torch to 1.x.


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v2.2.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../root/.cache/torch/pyannote/models--pyannote--embedding/snapshots/4db4899737a38b2d618bbd74350915aa10293cb2/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.2.0. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.2.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [ ]:
from safetensors.torch import save_file


# TODO cycle through files and create safe tensors of the embeddings and save them somewhere
def run_pre_process(root_directory, output_path):
    for subdir, dirs, files in tqdm(os.walk(rootdir)):
        for f in files:
            embedding = pre_process_new(os.path.join(subdir, f), separator, embedding_model)
            torch_embedding = torch.from_numpy(embedding)
            data = {
                    "tensor": torch_embedding,
                }
            # sub = subdir.split("/")[-2]
            fname = f.split(".")[0]
            output_file_name = f"{fname}.safetensors"
            save_file(data, output_path + output_file_name)


In [ ]:
# MUST delete list file
rootdir = "/content/ai_vocal_id_files"
output_dir = "ai_vocal_id_tensors/"
run_pre_process(rootdir, output_dir)

0it [00:00, ?it/s]WARNING:tensorflow:From /usr/local/lib/python3.10/dist-packages/spleeter/separator.py:146: calling DatasetV2.from_generator (from tensorflow.python.data.ops.dataset_ops) with output_types is deprecated and will be removed in a future version.
Instructions for updating:
Use output_signature instead
Instructions for updating:
Use output_signature instead
Instructions for updating:
Colocations handled automatically by placer.
1it [04:46, 286.52s/it]


In [ ]:
!gsutil -m cp -r ai_vocal_id_tensors/ gs://singer-id

Copying file://ai_vocal_id_tensors/YNy8VqlPX6Q.safetensors [Content-Type=application/octet-stream]...
Copying file://ai_vocal_id_tensors/uQnn0aRTnJc.safetensors [Content-Type=application/octet-stream]...
Copying file://ai_vocal_id_tensors/IQscAY8-9vg.safetensors [Content-Type=application/octet-stream]...
Copying file://ai_vocal_id_tensors/acEXM41m9mM.safetensors [Content-Type=application/octet-stream]...
Copying file://ai_vocal_id_tensors/mF1PkR8uZ2Y.safetensors [Content-Type=application/octet-stream]...
Copying file://ai_vocal_id_tensors/C0Mja5SdZEk.safetensors [Content-Type=application/octet-stream]...
Copying file://ai_vocal_id_tensors/PWQG3mwH8i8.safetensors [Content-Type=application/octet-stream]...
Copying file://ai_vocal_id_tensors/Lr8HJ3oW9-8.safetensors [Content-Type=application/octet-stream]...
Copying file://ai_vocal_id_tensors/dcGKHNDRHkw.safetensors [Content-Type=application/octet-stream]...
Copying file://ai_vocal_id_tensors/5BfB3YRzMwo.safetensors [Content-Type=applicati